In [4]:
import numpy as np

In [1]:
import numpy as np
import os, re, csv, math, codecs
from keras.models import Sequential
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer as t
from tqdm import tqdm
from keras.layers import merge, Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.models import *
# to visualize, and to make zero shape matrix
from attention_utils import get_activations, get_data_recurrent
from Attention import Attention
import pandas as pd
from konlpy.tag import Okt as Twitter
from selfword2vec import tokenization
from Anomaly import checkAnomaly_x_y
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from numpy import argmax
from keras import backend as K


# 1. 모델 저장시키기 
# 2. tokenizer와 konlpy morphs 호환 여부 (완성 jupyter note)
# 2'. word2vec 2가지 경우 더 추가 to embedding(final_total word2vec, twitter_translated.vec)
# 3. 변수들 설정하기


np.random.seed(3)

# NUM_WORDS, train으로 input받은 단어의 수
MAX_NB_WORDS = 20000
vocab_size = 0
EMB_DIM = 300
embeddings_index = dict()

# columns = ["ID", "Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism","sadness","surprise","trust"]
columns = ["ID","Tweet","분노","기대","혐오스러운","두려움","기쁨","사랑","낙관론","비관론","슬픔","놀라움","믿음"]




##########################################1. 데이터셋 생성하기

train_array = pd.read_csv("/home/minwookje/coding/ex1_w2v/data/tweet/dump/kor_train.txt",sep="\t", header=None,names=columns).values
val_array = pd.read_csv("/home/minwookje/coding/ex1_w2v/data/tweet/dump/kor_dev.txt",sep="\t", header=None,names=columns).values
test_array = pd.read_csv("/home/minwookje/coding/ex1_w2v/data/tweet/dump/kor_test_gold.txt",sep="\t", header=None,names=columns).values

# 판다 shape
print("train_array"+ str(train_array.shape))
print("val_array"+str(val_array.shape))
print("test_array"+str(test_array.shape))

print("Reading data!")
#  x, y 분할하기
x_train = train_array[1:,1]
y_train = train_array[1:,2:]
x_val = val_array[1:,1]
y_val = val_array[1:,2:]
x_test = test_array[1:,1]
y_test = test_array[1:,2:]


print("checking Anomaly!!")
x_train, y_train = checkAnomaly_x_y(x_train,y_train)
x_val, y_val = checkAnomaly_x_y(x_val,y_val)
x_test, y_test = checkAnomaly_x_y(x_test,y_test)

print("ANomaly result!")
print("x_train.shape"+ str(x_train.shape))
print("y_train.shape"+ str(y_train.shape))
print("x_val.shape"+ str(x_val.shape))
print("y_val.shape"+ str(y_val.shape))
print("x_test.shape"+ str(x_test.shape))
print("y_test.shape"+ str(y_test.shape))
# print(type(x_train)) np.array로 변형이 필요한가
print("Finished!")

# jupyter notebook
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------

tmp = []
train_tmp = []
val_tmp = []
test_tmp = []
max_count = 0
set_words = set()

print("Tokenizing!")
# test의 tmp만 따로 받아주고 나머지는 val train test모두 통합시켜준다.
test_tmp, dummy , set_words = tokenization(x_test)
val_tmp, dummy , set_words = tokenization(x_val)
train_tmp, max_count , set_words = tokenization(x_train)
tmp, dummy , set_words = tokenization(np.hstack([x_train,x_val,x_test]))
print("Tokenizing finished!")
tmp = [] # tmp없애준다



#token shape
print("It's by len()")
print("train_tmp.len():" + str(len(train_tmp))+","+ str(len(train_tmp[0])))
print("val_tmp shape.len():" + str(len(val_tmp))+","+ str(len(val_tmp[0])))
print("test_tmp shape.len():" + str(len(test_tmp))+","+ str(len(test_tmp[0])))


print("max_count:"+str(max_count))
# 문장길이 100으로 맞춘다.
max_count = min(100, max_count)


print("Readding Embedding file")
# embeddings_index == dict(w2v's word: vector)
f = codecs.open('/home/minwookje/coding/ex1_w2v/embedding/1542954106final_total_pos.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


# word2index table 생성 {token:index}
# 모든 train val test에 사용되는 token별 index table
# train 문장 token 갯수
word_index = len(set_words)
word_tmp_dict = dict()
for i, word in enumerate(set_words):
    word_tmp_dict[str(word).replace(" ", "")] = i
word_tmp_dict['0'] = word_index 

## 문장별 토큰화시킨 녀석에 index를 집어 넣어준다. 이때 pad도 동시에 해준다. 
word_vec = []
word_vec_test = []
word_vec_val = []

for sent in train_tmp:
    sub = []
    for word in sent:
        if(len(sub)==max_count):
            break    
         #print(word)
         #print(type(str(word)))
         #break
# word는 tuple 타입, embeddings_index는 str타입, tuple 타입을 str()화시키면 
# 중간에 space가 생성되어 match가 되지 않았다. 이를 해결해주었다. 
        if(str(word).replace(" ", "") in word_tmp_dict):
            
            sub.append(word_tmp_dict[str(word).replace(" ", "")])
        else:
            print("sentence index화 실패")
    count = max_count - len(sub)
    # padding
    sub.extend([word_index]*count)
    word_vec.append(sub)
## 테스트용 복사본 
for sent in test_tmp:
    sub = []
    for word in sent:
        if(len(sub)==max_count):
            break
         #print(word)
         #print(type(str(word)))
         #break
# word는 tuple 타입, embeddings_index는 str타입, tuple 타입을 str()화시키면 
# 중간에 space가 생성되어 match가 되지 않았다. 이를 해결해주었다. 
        if(str(word).replace(" ", "") in word_tmp_dict):
            
            sub.append(word_tmp_dict[str(word).replace(" ", "")])
        else:
            print("sentence index화 실패")
    count = max_count - len(sub)
    # padding
    sub.extend([word_index]*count)
    word_vec_test.append(sub)

## 검증용 복사본 
for sent in val_tmp:
    sub = []
    for word in sent:
        if(len(sub)==max_count):
            break
         #print(word)
         #print(type(str(word)))
         #break
# word는 tuple 타입, embeddings_index는 str타입, tuple 타입을 str()화시키면 
# 중간에 space가 생성되어 match가 되지 않았다. 이를 해결해주었다. 
        if(str(word).replace(" ", "") in word_tmp_dict):
            
            sub.append(word_tmp_dict[str(word).replace(" ", "")])
        else:
            print("sentence index화 실패")
    count = max_count - len(sub)
    # padding
    sub.extend([word_index]*count)
    word_vec_val.append(sub)

print("word_vec shape:" + str(len(word_vec))+","+ str(len(word_vec[0])))
print("word_vec_val shape:" + str(len(word_vec_val))+","+ str(len(word_vec_val[0])))
print("word_vec_test shape:" + str(len(word_vec_test))+","+ str(len(word_vec_test[0])))





Using TensorFlow backend.


train_array(6651, 13)
val_array(887, 13)
test_array(3239, 13)
Reading data!
checking Anomaly!!
(6619,)
(6619, 11)
(883,)
(883, 11)
(3215,)
(3215, 11)
ANomaly result!
x_train.shape(6619,)
y_train.shape(6619, 11)
x_val.shape(883,)
y_val.shape(883, 11)
x_test.shape(3215,)
y_test.shape(3215, 11)
Finished!
Tokenizing!


589it [00:00, 5885.52it/s]

Tokenizing finished!
It's by len()
train_tmp.len():6619,40
val_tmp shape.len():883,39
test_tmp shape.len():3215,46
max_count:10531
Readding Embedding file


45942it [00:05, 8058.71it/s]


Loaded 45942 word vectors.
word_vec shape:6619,100
word_vec_val shape:883,100
word_vec_test shape:3215,100


In [4]:
# 4번쨰 matrix embedding_matrix {index: vector}
# vocab_size = min(MAX_NB_WORDS, word_index)
vocab_size = word_index
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size+1, EMB_DIM))
match_count = 0
unmatch_count = 0

for word, i in word_tmp_dict.items():
    if word != '0':
        if (word in embeddings_index):
            match_count += 1
            embedding_matrix[i] = embeddings_index[word]
        else:
            unmatch_count += 1
            embedding_matrix[i] = np.random.uniform(-0.25,0.25,300) ## used for OOV words

print("match:" + str(match_count))
print("unmatch:" + str(unmatch_count))

match:11300
unmatch:12626


In [5]:
word_vec = np.array(word_vec)
word_vec_val = np.array(word_vec_val)
word_vec_test = np.array(word_vec_test)
embedding_matrix = np.matrix(embedding_matrix)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [ ]:
INPUT_DIM = 300 #wordvec사이즈
max_count = max_count #문장의 max길이
lstm_shape = 250
rate_drop_lstm = 0.0
rate_drop_dense = 0.0

k_vec = word_vec
k_vec_val = word_vec_val
k_vec_test = word_vec_test
k_y_train = y_train
k_y_val = y_val
k_y_test = y_test
k_embedding_matrix = embedding_matrix

# 32하니까 죽어버린다.
batch_size = 512
# Define the model
# inp = InputLayer(shape=(max_count,), dtype='float32',sparse=True)
# input layer는 
# inp = InputLayer(input_shape=(max_count,),sparse=True)

# inp = Input(shape=(max_count,))
inp = Input(shape=(max_count,), dtype='float32')
emb = Embedding(input_dim=vocab_size+1, output_dim=EMB_DIM,
            trainable=False, weights=[embedding_matrix], input_length=max_count)(inp)
# max_features = vocab_size, maxlen=text_max_words, embed_size=EMB_DIM
# emb = Embedding(input_dim=max_features, input_length = maxlen, output_dim=embed_size)(inp)
# embedding dropout = 0.1
x = SpatialDropout1D(0.1)(emb)
x = Bidirectional(LSTM(lstm_shape,return_sequences=True, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))(emb)

x, attention = Attention()(x,rate_drop_dense)
x = Dense(11, activation="sigmoid")(x)
# x = Dense(11)(x)
model = Model(inputs=inp, outputs=x)
# accu = jaccard_distance_acc   
# # loss = tf.nn.sigmoid_cross_entropy_with_logits
# loss = binary_crossentropy
from keras import optimizers
model.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])
# attention_model = Model(inputs=inp, outputs=attention) # Model to print out the attention data
print(model.summary())
# # verbose= ? , validation_split은 validation file로 변환시켜주어야 한다.






# STAMP = './First_AttentionNLP_%.2f_%.2f'%(rate_drop_lstm,rate_drop_dense)
# print(STAMP)
# early_stopping =EarlyStopping(monitor='val_loss', patience=5)
# bst_model_path = STAMP + '.h5'
# model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
# # model.fit(X_t, y, validation_data=(x_val,y_val), epochs=3, verbose=1, batch_size=512)
# epoch = 50

model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=1, verbose=1, batch_size=batch_size)


# model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=3, verbose=1, steps_per_epoch=int(6619/batch_size)+1, validation_steps = int(883/batch_size)+1)

# 5. 모델 평가하기
score, acc = model.evaluate(k_vec_test, k_y_test, batch_size=batch_size)
print('')
print('score : ' + str(score)+ 'acc : ' + str(acc))



In [12]:
result = model.predict(k_vec_val, batch_size=batch_size, verbose=0)
for value in result:
    print(value)

[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
[ nan  nan  nan  nan  nan  nan 

In [23]:
layer_name = 'my_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer('tanh_mlp2').output)
intermediate_output = intermediate_layer_model.predict(k_vec_val)

In [22]:
print(model.input)

Tensor("input_5:0", shape=(?, 100), dtype=float32)


In [24]:
print(model.input)

Tensor("input_5:0", shape=(?, 100), dtype=float32)


In [26]:
from keras import backend as K

# with a Sequential model
get_3rd_layer_output = K.function([model.layers[1].input],
                                  [model.layers[2].output])
layer_output = get_3rd_layer_output([x])

InvalidArgumentError: Requested Tensor connection between nodes "input_5" and "dense_4/Sigmoid" would create a cycle.